In [66]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid="1653774166%3A9uDdOCkxtjfjYr%3A25%3AAYf2jGK4vIx5W_dkE5m8a7PpoN4LA2D5CxUpR4C7BA")
t = RetrieveDataByQuery(sessionid="65955050144%3ACYLJ62xy3SlmZY%3A21%3AAYczdKeVZcK3hiLzXoCohMHegWecE3FIvmjZ_Myz8w")
t = RetrieveDataByQuery(sessionid="71441509544%3A9NroGpYJCtlrjn%3A1%3AAYe0D7Oa0fAH8aWyYbNn37a64dbS-vNqOjYqoxqPtA")

In [67]:
query = "Artificial Intelligence"

In [68]:
import threading
import pandas as pd

def get_follower_data(data:list):
    result = []
    for d in data:
        node = d.get("node", {})
        result.append({
            "pk": node.get("pk", ""),
            "username": node.get("username", ""),
            "follower_count": node.get("follower_count", "")
        })
    return result

def fetch_followers_thread(userid: str, i: int, results: list, lock: threading.Lock):
    after = str(25 * i)
    response = t.retrieve_follower_by_userid_after(userid, after)
    data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
    batch_results = get_follower_data(data.get("edges", []))
    
    with lock:
        results.extend(batch_results)
        
def get_all_follower(userid: str):
    results = []
    response = t.retrieve_follower_by_userid(userid)
    data = response.get("data", {}).get("user", {}).get("followers", {})
    results.extend(get_follower_data(data.get("edges", [])))
    
    has_next_page = data.get("page_info", {}).get("has_next_page", False)
    i = 1
    lock = threading.Lock()
    threads = []
    
    while has_next_page:
        print(i)
        thread = threading.Thread(target=fetch_followers_thread, args=(userid, i, results, lock))
        threads.append(thread)
        thread.start()
        i += 1
        
        response = t.retrieve_follower_by_userid_after(userid, str(25 * i))
        data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
        has_next_page = data.get("page_info", {}).get("has_next_page", False)
    
    for thread in threads:
        thread.join()
    
    return results



def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [69]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [70]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [71]:
import json
import pandas as pd
df = pd.read_csv("data.csv")
df['user_pk'] = df['user_pk'].astype(str)
unique_user_pk = df['user_pk'].unique()
len(unique_user_pk)

874

In [ ]:
import pandas as pd
import os
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())

def fetch_followers(pk):
    logging.info(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None

def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")

csv_file = "followers_data.csv"
processed_users = load_processed_users(csv_file)

follower_list = []
batch_size = 2
i = 0

logging.info("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        logging.info(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    i += 1
    
    if i >= batch_size:
        logging.info(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []  
        i = 0

if follower_list:
    logging.info(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

logging.info("Follower data collection completed")

In [ ]:
df1 = pd.read_csv("followers_data.csv")
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
dfcount

(17, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
187264532,217,217,217
241752404,49,49,49
273811002,167,167,167
295418393,1530,1530,1530
429303617,264,264,264
623659295,627,627,627
1175659953,247,247,247
1624593034,50,50,50
3312160968,433,433,433
